First things first, let's get our dependencies all set!

In [ ]:
!pip install openai langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 5.2 MB/s eta 0:00:00


this loads our PDF document Ultimate Tech Resume Guide by JS Mastery

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/capstone/Resume_guide.pdf")
pages = loader.load_and_split()

In [ ]:
#An advantage of this approach is that documents can be retrieved with page numbers.

pages[0]

Document(page_content='TECH\nRESUME\nGUIDE\nSoftware\nengineers\nknow\nthat\nthere\nis\nno\none\nfit-size\nsolution\nfor\nanything.\nBe \nthat\ndeveloping\na\nwebsite,\nsolving\nan\nalgorithm,\nor\ndesigning\nan\narchitecture. \nWhile\nthis\nguide\npromises\nto\ndeliver\ngreat\ncontent\nto\nhelp\nyou\nace\na\njob\ninterview, \nit\ndoesn’t\nguarantee\na\njob\nupfront\nor\na\nrecruiter\ncall\nevery\ntime.\nThere\nare\nplenty\nof\nfactors\nthat\ncontribute\nto\njob\nhunting.\nWhile\nfew\nare\nin\nhands, \nfew\naren’t.\nIt’s\nthat\ndead\nsimple.\nSo\nwhat\nare\nthe\nfactors\nthat\nyou\nneed\nto\ntake \ncare\nof?\nThere\nare\nplenty\nof\nfactors\nthat\ncontribute\nto\njob\nhunting.\nWhile\nfew\nare\nin\nhands,\nfew\naren’t.\nIt’s\nthat\ndead\nsimple.\nSkills\nYou\nmust\nhave\nrelevant\nskills\nto\napply\nfor\na\nspecific\njob.\nExperience\nSome\nindustry\nexperience\ncan\nmake\nyour\nresume\nstand\nout\nmore.\nCommunication\nYou\nshould\nat\nleast\ncommunicate\nwell\nin\nEnglish\n(as\nits\n

Now that we have our data loaded we can move onto splitting those documents into appropriately sized pieces!

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)

documents = text_splitter.split_documents(pages)

In [ ]:
documents

[Document(page_content='TECH\nRESUME\nGUIDE\nSoftware\nengineers\nknow\nthat\nthere\nis\nno\none\nfit-size\nsolution\nfor\nanything.\nBe \nthat\ndeveloping\na\nwebsite,\nsolving\nan\nalgorithm,\nor\ndesigning\nan\narchitecture. \nWhile\nthis\nguide\npromises\nto\ndeliver\ngreat\ncontent\nto\nhelp\nyou\nace\na\njob\ninterview, \nit\ndoesn’t\nguarantee\na\njob\nupfront\nor\na\nrecruiter\ncall\nevery\ntime.\nThere\nare\nplenty\nof\nfactors\nthat\ncontribute\nto\njob\nhunting.\nWhile\nfew\nare\nin\nhands, \nfew\naren’t.\nIt’s\nthat\ndead\nsimple.\nSo\nwhat\nare\nthe\nfactors\nthat\nyou\nneed\nto\ntake \ncare\nof?\nThere\nare\nplenty\nof\nfactors\nthat\ncontribute\nto\njob\nhunting.\nWhile\nfew\nare\nin\nhands,\nfew\naren’t.\nIt’s\nthat\ndead\nsimple.\nSkills\nYou\nmust\nhave\nrelevant\nskills\nto\napply\nfor\na\nspecific\njob.\nExperience\nSome\nindustry\nexperience\ncan\nmake\nyour\nresume\nstand\nout\nmore.\nCommunication\nYou\nshould\nat\nleast\ncommunicate\nwell\nin\nEnglish\n(as\nits\

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Now we can embed our chunked documents into a ChromaVectorDB!

We are going to be going through the example of persisting this particular DB - but we won't be leveraging the persisted version.

In [ ]:
# we'll need a few dependencies before we can do this
!pip install chromadb tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from langchain.vectorstores import Chroma

persist_directory = "vector_db"

vectordb = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=persist_directory)

In [ ]:
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [ ]:
doc_retriever = vectordb.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA

resume_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever)

In [ ]:

#https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/

from langchain.prompts import PromptTemplate
prompt_template = """You are a resume expert. You will be given a resume or both a
resume and a job description, and any special circumstances. You will first determine
the experience level (such as entry-level, jr, or senior) of the candidate
and what type of job they should apply to such as data scientist, full-stack engineer,
dentist, etc. You will then generate a report on their resume based on best practices
and their experience level and situation, and special circumstances.
Be sure to address their special circumstances, if any. If a job description is given
you should also include helpful information about how to optimize their resume to that
specific job. Lastly, you will apply all the recommendations
you made in the report to generate a optimized resume. You will not make up any information
about the candidate in the optimized version of the resume. Optmize the resume even if
they are not a good fit, assume that they are changing careers and give advice to that effect.
You may include recommendations not included as long as you make it clear you included
them and denote the recommendations you made via ***recommendations***.

Special circumstance(s): {special}

Resume: {resume}

Job Description: {job_description}

Output:"""


In [ ]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["resume", "job_description", "special"]
)

In [ ]:
special_1 = 'none'
special_2 = 'switching careers'
special_3 = 'career break. For context, today is %s' %today

In [ ]:
resume = '''Personal Information:

John Doe
123 Main St, City, State 12345
Phone: (123) 456-7890
Email: john.doe@example.com

Objective:

A budding data scientist with an academic background in Statistics and Computer Science, looking to leverage my skills in data analysis, machine learning, and programming at a progressive organization that encourages innovative thought.

Education:

Bachelor of Science in Computer Science, Minor in Statistics
University XYZ
August 2019 - May 2023

Skills:

Programming Languages: Python, R, SQL
Libraries: NumPy, Pandas, Matplotlib, Seaborn, Scikit-learn
Tools: Jupyter Notebooks, Tableau, Git
Machine Learning: Regression, Classification, Clustering, Neural Networks
Strong analytical and problem-solving skills
Projects:

Predictive Analysis of Airline Customer Satisfaction: Utilized machine learning algorithms to predict customer satisfaction for an airline company. Cleaned and analyzed a dataset of over 10,000 entries, implemented multiple regression models and improved prediction accuracy by 15%.

Analyzing and Visualizing NYC 311 Calls Data: Performed data cleaning, analysis, and visualization on a dataset of over 1 million 311 calls. Presented findings in an interactive dashboard using Tableau.

Work Experience:

Data Analyst Intern - Company ABC, City, State
June 2023 - Present

Assisting in data preprocessing, analysis, and generating insightful reports for decision-making.
Implemented machine learning models to predict future sales, improving forecast accuracy by 10%.
Certifications:

Data Science Certification – Coursera
Certified: Python for Data Science – edX
References:

Available upon request'''

In [ ]:
job_description ='''
Qualifications
•The ideal candidate should possess a strong analytical mindset, excellent problem-solving abilities, and a passion for translating data into actionable strategies
•Bachelor's degree in Computer Science, Statistics, Mathematics, or a related field
•5+ years proven experience as a Data Scientist or a related role, with a strong track record of delivering impactful data-driven solutions
•Proficiency in programming languages such as Python, R, or Java, along with experience using data manipulation and analysis libraries (e.g., Pandas, NumPy, scikit-learn)
•Solid understanding of statistical analysis, machine learning algorithms, and experimental design
•Proficiency in SQL and experience working with relational databases (e.g., MySQL, PostgreSQL)
•Strong data visualization skills using tools such as Tableau, Power BI, or D3.js
•Excellent problem-solving abilities and a keen attention to detail
•Strong written and verbal communication skills with the ability to effectively present complex findings to non-technical stakeholders
•Ability to work independently and collaborate effectively in a team-oriented environment
•Strong organizational skills and the ability to manage multiple projects and priorities
•McAllen, TX 78501: Reliably commute or planning to relocate before starting work (Required)
Responsibilities
•As a Data Scientist, you will play a critical role in extracting actionable insights from complex data sets to enhance business decision-making and drive organizational growth
•You will collaborate with cross-functional teams to identify data-driven opportunities, develop and implement advanced analytical models, and deliver compelling visualizations and reports
•Collaborate with stakeholders to identify business objectives and data-driven opportunities
•Develop and implement cutting-edge predictive and prescriptive models using statistical analysis and machine learning techniques
•Clean, preprocess, and analyze large and complex data sets to uncover hidden patterns and insights
•Create robust data pipelines and automate data extraction, transformation, and loading (ETL) processes
•Design and execute experiments to test hypotheses, analyze results, and provide actionable recommendations
•Develop and deploy data-driven dashboards, reports, and visualizations to communicate findings effectively
•Stay updated with the latest industry trends and advancements in data science techniques and tools
•Present findings and recommendations to key stakeholders in a clear and concise manner
•Provide guidance and mentorship to junior team members, as needed
Benefits
•Competitive salary commensurate with experience
•Comprehensive health, dental, and vision insurance plans
•Retirement savings plan
•Paid vacation, holidays, and sick leave
•Professional development and training opportunities
•Pay: From $120,000.00 per year
•Flexible schedule
•Paid time off
•Parental leave
•Relocation assistance
•Supplemental pay types:
•Bonus pay
'''

In [ ]:
response_1 = resume_qa(PROMPT.format(resume=resume, job_description=job_description, special = special_1))


In [ ]:
print(response_1['result'])

Based on the provided resume, the candidate appears to be an entry-level data scientist or data analyst. They have a strong academic background in Statistics and Computer Science, as well as relevant skills in programming languages, machine learning, and data analysis.

To optimize the resume for data scientist positions, here are some recommendations:

1. Objective: The objective statement is well-written and highlights the candidate's skills and goals. No changes are necessary.

2. Education: The education section is well-presented, mentioning the degree, university, and expected graduation date. No changes are necessary.

3. Skills: The skills section is comprehensive and includes relevant programming languages, libraries, tools, and machine learning techniques. No changes are necessary.

4. Projects: The projects section showcases the candidate's practical experience in data analysis and machine learning. However, it would be beneficial to provide more details about the methodologi

In [ ]:
job_description_2 = '''
Qualifications
•Bachelor's degree in an engineering discipline
•Experience with at least one commercial FEA or multiphysics simulation package (e.g
•Experience with mechanical design and analysis (internship experience qualifies)
•Python, Matlab, Excel)
•Self-motivated with strong organizational, written, and oral communication skills
•Analytical/scientific programming experience (i.e
•Ability to work extended hours and weekends as necessary
•To conform to U.S. Government export regulations, applicant must be a (i) U.S. citizen or national, (ii) U.S. lawful, permanent resident (aka green card holder), (iii) Refugee under 8 U.S.C. § 1157, or (iv) Asylee under 8 U.S.C. § 1158, or be eligible to obtain the required authorizations from the U.S
Responsibilities
•Own primary and secondary structures on Starship
•Design flight hardware to achieve ascent, entry, and rapid reuse of next generation launch vehicle
•Develop design criteria where definition is lacking
•Perform hand calculations and FEA to develop, assess, and optimize designs
•Incorporate design for manufacturing principles and supervise first article builds
•Balance operational needs across vehicle systems while optimizing for development speed, mass and cost
•Design and execute qualification testing
Benefits
•Mechanical Engineer/Level I: $95,000.00 - $115,000.00/per year
•Mechanical Engineer/Level II: $110,000.00 - $130,000.00/per year
•Base salary is just one part of your total rewards package at SpaceX
•You may also be eligible for long-term incentives, in the form of company stock, stock options, or long-term cash awards, as well as potential discretionary bonuses and the ability to purchase additional stock at a discount through an Employee Stock Purchase Plan
•You will also receive access to comprehensive medical, vision, and dental coverage, access to a 401(k) retirement plan, short and long-term disability insurance, life insurance, paid parental leave, and various other discounts and perks
•You may also accrue 3 weeks of paid vacation and will be eligible for 10 or more paid holidays per year
•Exempt employees are eligible for 5 days of sick leave per year
'''

In [ ]:
response_2 = resume_qa(PROMPT.format(resume=resume, job_description=job_description_2, special = special_2))

In [ ]:
print(response_2['result'])


Based on the resume provided, the candidate is a budding data scientist with a background in Statistics and Computer Science. They have relevant skills in programming languages such as Python, R, and SQL, as well as experience with libraries and tools commonly used in data analysis and machine learning. They have also completed projects that demonstrate their ability to apply their skills in real-world scenarios.

Considering the candidate's special circumstance of switching careers, it is important to highlight their transferable skills and showcase how their previous experience and education can be valuable in a data science role. Here are some recommendations for optimizing the resume:

1. Objective: The objective statement should be revised to emphasize the candidate's transferable skills and their motivation to transition into a data science role. For example:

Objective: A highly motivated and analytical professional with a strong background in Statistics and Computer Science, se

In [ ]:
resume_3 = '''Personal Information:

Jane Doe
123 Main St, City, State 12345
Phone: (123) 456-7890
Email: jane.doe@example.com

Objective:

Accomplished data scientist with over 7 years of experience in utilizing data-driven decision making to solve complex business problems. Looking to leverage my extensive background in machine learning, data mining, and predictive modeling in a challenging role.

Education:

Master of Science in Data Science
XYZ University
August 2010 - May 2012

Skills:

Programming Languages: Python, R, SQL, Java
Libraries: NumPy, Pandas, Matplotlib, Seaborn, Scikit-learn, TensorFlow
Tools: Jupyter Notebooks, Tableau, Apache Spark, Hadoop
Machine Learning: Deep Learning, Reinforcement Learning, Natural Language Processing
Strong leadership and project management skills
Projects:

Fraud Detection System: Designed and developed a fraud detection system using machine learning models which reduced the fraudulent transactions by 30%.

Predictive Maintenance of Industrial Machines: Implemented predictive analytics to anticipate the failure of machine parts which decreased the maintenance cost by 20%.

Work Experience:

Senior Data Scientist - Company ABC, City, State
June 2016 - May 2020

Led a team of 5 data scientists to manage machine learning and data science projects from conception to execution.
Developed predictive models using machine learning algorithms to drive strategic decisions, resulting in a 25% increase in marketing campaign effectiveness.
Data Scientist - Company XYZ, City, State
May 2012 - June 2016

Managed and analyzed big data, implemented machine learning models for various business needs.
Assisted in creating a recommendation engine for the e-commerce platform, resulting in a 20% increase in sales.
Certifications:

Certified Data Scientist – Data Science Council of America (DASCA)
Advanced SQL for Data Scientists – Coursera
References:

Available upon request'''

In [ ]:
job_description_3 = '''Work with Commercial Analytics Research & Development team to establish data science capabilities within CITGO. Design, develop and implement data-driven solutions that help the organization make better decisions, and drive the adoption of data-driven decision making within the organization. The role requires a combination of technical skills in data science, software engineering and machine learning, as well as communication and project management skills.

Analysis is concerned with three primary areas: (1) Optimization Solutions, such as measurement of competitive performance and its drivers, customer/product mix, sales lift studies, price elasticities, etc., (2) Predictive & Exploratory Analytics, such as demand forecasting, prioritization of trends, market potential for new business models, (3) Thought Leadership, such as applying augmented decision making with ML/AI, creative use of internal and external data sources with pragmatic curiosity.

Individual will act as an individual contributor within a group of technical personnel.

Individual will function in the capacity of a full stack decision scientist to address majority of initial hard data science needs as data science team begins establishment.

Individual will develop and implement predictive modelling to optimize profitability and decision-making capabilities.

Individual will collaborate with stakeholders to understand business needs and decision-making processes.

Requires very strong understanding of the business drivers of decision making and communicating complex insights to a diverse audience.

Requires application of well-developed technical knowledge in data science, conducting and coordinating difficult, responsible assignments.

Minimum Qualifications:

Degree:
• Bachelor’s Degree.
• Masters or PhD degree in computer science, statistics, data science or similar strongly preferred.

The minimun number of years of job related experience required by this job is:
• 8-10+ years.

List any specialized training or unique skills required:
• Deep understanding of data science/decision science concepts and techniques, as well as hands on experience building data-driven solutions using machine learning algorithms, statistical models and data visualization tools, required.
• Experience with major cloud computing platforms (Azure, AWS, etc.) required.
• Experience with Azure platforms strongly preferred.
• Proficiency in data analytics tools such as Python, R, SQL preferred.
• Understanding of downstream oil & gas business preferred.
• Ability to effectively manage project timelines, budgets and resources is necessary.
• Excellent written and verbal communication skills preferred.
• Strong knowledge of data privacy, security and governance regulations/best practices.
• Ability to simplify complex information.
• Desire and willingness to continuously expand knowledge and skills.
• Passionate and creative about solving complex and difficult problems, both in group and individual settings
• Applicant must be self-starter and comfortable with situations where solutions are not always clear or obvious.

Job Duties:
• Develop & implement data-driven solutions to support decision making, such as predictive models, data visualization dashboards and machine learning algorithms. Communicate the results of data science projects to key stakeholders and executive leadership, promoting the adoption and integration of data science into decision-making processes.
• Collaborate with Commercial Analytics R&D & business units to identify and prioritize data science projects that can deliver business value. This may involve conducting data analysis and data discovery activities to understand the current state of the data assets and how they can be leveraged to support decision making.
• Collaborate with Commercial Analytics R&D team to establish the necessary strategies, technical infrastructure and tools to support data science activities, including data storage, data processing and machine learning. Approaches for this should have a heightened focus on approaches that are innovative, organizationally sustainable, modern, transparent and efficient.
• Ensure that data privacy, security and governance standards are met in all data science projects. Work with IT and Commercial Analytics R&D to implement appropriate measures for data collection, processing, storage and protection. Ensure that data is of high quality, properly documented, and stored and managed in a manner that meets regulatory and organizational requirements.
• Stay current on new techniques, technologies and tools in data science, machine learning and the oil and gas industry and evaluate and determine their potential applications for the organization. Requires a commitment to ongoing professional development and a willingness to continuously expand expertise, which includes attending regular training, industry events, reading trade publications and engaging with colleagues and peers in the field.'''

In [ ]:
response_3 = resume_qa(PROMPT.format(resume=resume_3, job_description=job_description_3, special = special_3))

In [ ]:
print(response_3['result'])

Based on the provided resume and job description, the candidate is an experienced data scientist with over 7 years of experience. They are seeking a challenging role that allows them to leverage their background in machine learning, data mining, and predictive modeling. The job description is for a position within CITGO, where the individual will be responsible for establishing data science capabilities and implementing data-driven solutions.

Here are some recommendations for optimizing the candidate's resume:

1. Objective: The objective statement should be tailored to the specific job description. In this case, the candidate can modify their objective to highlight their interest in establishing data science capabilities within CITGO and driving data-driven decision making.

***Recommendation: Modify the objective statement to align with the job description and emphasize the candidate's interest in establishing data science capabilities within CITGO.***

2. Skills: The candidate's sk

In [ ]:
result_3 = response_3['result']

In [ ]:
from datetime import date

# Get today's date
today = date.today()

In [ ]:
refine_template = '''
You are a resume expert revising the work of one of your peers. Make sure the output makes sense
based on best practices.
Revise your peer's work and consider and special circumstances the applicant might have.
If your peer did not address the applicant's special circumstance, include it in a new report
and new optimized resume. Rerender the report and revise the work of the peer.

This is your peer's work: {result}

Special circumstances: {special}

For reference today is {today}. Make sure there are no career gaps.

Make sure to output the resume as a python dictionary, using the curly braces.


'''



In [ ]:
refined_PROMPT = PromptTemplate(
    template=refine_template, input_variables=["result", "special", "today"])

In [ ]:
refined_response_3 = resume_qa(refined_PROMPT.format(result= result_3, special = special_3, today =today))

In [ ]:
rr_3 = refined_response_3['result']

In [ ]:
rr_3

'{\n  "Personal Information": {\n    "Name": "Jane Doe",\n    "Address": "123 Main St, City, State 12345",\n    "Phone": "(123) 456-7890",\n    "Email": "jane.doe@example.com"\n  },\n  "Objective": "Accomplished data scientist with over 7 years of experience in utilizing data-driven decision making to solve complex business problems. Looking to leverage my extensive background in machine learning, data mining, and predictive modeling to establish data science capabilities within CITGO and drive data-driven decision making.",\n  "Education": {\n    "Degree": "Master of Science in Data Science",\n    "University": "XYZ University",\n    "Dates": "August 2010 - May 2012"\n  },\n  "Skills": {\n    "Programming Languages": "Python, R, SQL, Java",\n    "Libraries": "NumPy, Pandas, Matplotlib, Seaborn, Scikit-learn, TensorFlow",\n    "Tools": "Jupyter Notebooks, Tableau, Apache Spark, Hadoop",\n    "Machine Learning": "Deep Learning, Reinforcement Learning, Natural Language Processing",\n    

In [ ]:
pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53898 sha256=df1eb200f57ce20813f40b14394aa29a61d3eac0115a14481d116d57fd9835b4
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [ ]:
pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184489 sha256=165ae04c482d81f65738a9cb958ae6a04ee2cd19cdbc2a11123225f0409842c5
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
from docx import Document
import json

def create_resume_doc(json_data, output_filename):
    # Create a new Word document
    doc = Document()

    # Loop through the JSON data and add sections to the document
    for section, content in json_data.items():
        doc.add_heading(section, level=2)
        if isinstance(content, dict):
            for key, value in content.items():
                doc.add_paragraph(f"{key}: {value}")
        elif isinstance(content, list):
            for item in content:
                if isinstance(item, dict):
                    for key, value in item.items():
                        doc.add_paragraph(f"{key}: {value}")
                else:
                    doc.add_paragraph(item)
        else:
            doc.add_paragraph(content)

    doc.save(output_filename)


In [ ]:
data_dict = json.loads(rr_3)


In [ ]:
data_dict

{'Personal Information': {'Name': 'Jane Doe',
  'Address': '123 Main St, City, State 12345',
  'Phone': '(123) 456-7890',
  'Email': 'jane.doe@example.com'},
 'Objective': 'Accomplished data scientist with over 7 years of experience in utilizing data-driven decision making to solve complex business problems. Looking to leverage my extensive background in machine learning, data mining, and predictive modeling to establish data science capabilities within CITGO and drive data-driven decision making.',
 'Education': {'Degree': 'Master of Science in Data Science',
  'University': 'XYZ University',
  'Dates': 'August 2010 - May 2012'},
 'Skills': {'Programming Languages': 'Python, R, SQL, Java',
  'Libraries': 'NumPy, Pandas, Matplotlib, Seaborn, Scikit-learn, TensorFlow',
  'Tools': 'Jupyter Notebooks, Tableau, Apache Spark, Hadoop',
  'Machine Learning': 'Deep Learning, Reinforcement Learning, Natural Language Processing',
  'Cloud Computing Platforms': 'Azure',
  'Other': 'Strong leaders

In [ ]:

create_resume_doc(data_dict, '/content/drive/MyDrive/capstone/resume.docx')